<a href="https://colab.research.google.com/github/albey-code/hippoabstraction/blob/main/events_tsv_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook serves to convert all event files in .txt format into .tsv format for subject 4. I will write a function to implement this for all *N* = 23 subjects from Garvert et al. (2017) later. Importantly, .tsv format is in line with Brain Imaging Data Structure (BIDS) conventions and better for open-science and reproducibility.

In [38]:
from google.colab import files
import zipfile, os, glob, io
import pandas as pd

# Run 1 Subject 04

In [39]:
uploaded = files.upload()

Saving subject4_run1.zip to subject4_run1 (1).zip


In [40]:
# Get the name dynamically
zip_filename = list(uploaded.keys())[0]

In [41]:
# Unzip
unzip_dir = "unzipped_run1"
os.makedirs(unzip_dir, exist_ok=True)
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

print(os.listdir(unzip_dir))  # Sanity check

['Subj_4_run_1_obj_8_patch_0.txt', 'Subj_4_run_1_obj_4_patch_1.txt', 'Subj_4_run_1_obj_4_patch_0.txt', 'Subj_4_run_1_obj_8_patch_1.txt', 'Subj_4_run_1_obj_6_patch_1.txt', 'Subj_4_run_1_obj_7_patch_1.txt', 'Subj_4_run_1_obj_2_patch_0.txt', 'Subj_4_run_1_obj_6_patch_0.txt', 'Subj_4_run_1_obj_9_patch_0.txt', 'Subj_4_run_1_obj_10_patch_0.txt', 'Subj_4_run_1_obj_7_patch_0.txt', 'Subj_4_run_1_obj_9_patch_1.txt', 'Subj_4_run_1_button.txt', 'Subj_4_run_1_obj_10_patch_1.txt', 'Subj_4_run_1_obj_2_patch_1.txt']


In [42]:
stim_rows = []
button_rows = []

In [43]:
for filepath in glob.glob(f"{unzip_dir}/*.txt"):
    filename = os.path.basename(filepath)

    if "button" in filename.lower():
        # Button file: one column called 'button'
        df = pd.read_csv(filepath, sep='\s+', skiprows=1, names=["button"])
        df["onset"] = df["button"]
        df["duration"] = 0.0
        df["trial_type"] = "button_press"
        button_rows.append(df[["onset", "duration", "trial_type"]])

    else:
        # Object stimulus files
        df = pd.read_csv(filepath, sep='\s+', skiprows=1, names=["object", "patch", "onset", "prevObject", "dist"])
        df["duration"] = 0.0

        def label_trial(row):
            if pd.isna(row["object"]):
                return "other"
            obj = int(row["object"])
            if obj in [2, 4, 6, 7, 8, 9, 10]:
                return f"object_{obj}"
            elif row["patch"] == 1:
                return "patch_all"
            elif row["patch"] == 0:
                return "no_patch_all"
            else:
                return "other"

        df["trial_type"] = df.apply(label_trial, axis=1)
        stim_rows.append(df[["onset", "duration", "trial_type"]])

In [44]:
# Merge both types of events
stim_events = pd.concat(stim_rows, ignore_index=True)
button_events = pd.concat(button_rows, ignore_index=True)
events = pd.concat([stim_events, button_events], ignore_index=True).sort_values("onset")

In [45]:
# Final preview
print(events["trial_type"].value_counts())

trial_type
object_10       48
object_2        46
object_7        46
object_4        45
object_6        43
object_8        42
button_press    41
object_9        40
Name: count, dtype: int64


In [46]:
events

,onset,duration,trial_type
134,1.470,0.0,object_2
219,3.640,0.0,object_7
87,5.670,0.0,object_6
45,8.750,0.0,object_4
193,12.811,0.0,object_10
...,...,...,...
218,1305.799,0.0,object_10
350,1310.500,0.0,button_press
282,1319.029,0.0,object_10
22,1321.129,0.0,object_8


In [47]:
# Save to new _events.tsv
events.to_csv("sub04_run1_events.tsv", sep="\t", index=False)

In [48]:
# Downloading them locally
#from google.colab import files

files.download("sub04_run1_events.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Run 2 Subject 04

In [49]:
uploaded = files.upload()

Saving subject4_run2.zip to subject4_run2 (1).zip


In [50]:
# Get the name dynamically
zip_filename = list(uploaded.keys())[0]

In [51]:
# Unzip
unzip_dir = "unzipped_run2"
os.makedirs(unzip_dir, exist_ok=True)
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

print(os.listdir(unzip_dir))  # Sanity check

['Subj_4_run_2_obj_7_patch_0.txt', 'Subj_4_run_2_obj_4_patch_1.txt', 'Subj_4_run_2_obj_8_patch_0.txt', 'Subj_4_run_2_obj_10_patch_0.txt', 'Subj_4_run_2_obj_9_patch_1.txt', 'Subj_4_run_2_obj_2_patch_0.txt', 'Subj_4_run_2_obj_8_patch_1.txt', 'Subj_4_run_2_button.txt', 'Subj_4_run_2_obj_9_patch_0.txt', 'Subj_4_run_2_obj_6_patch_1.txt', 'Subj_4_run_2_obj_10_patch_1.txt', 'Subj_4_run_2_obj_6_patch_0.txt', 'Subj_4_run_2_obj_2_patch_1.txt', 'Subj_4_run_2_obj_4_patch_0.txt', 'Subj_4_run_2_obj_7_patch_1.txt']


In [52]:
stim2_rows = []     # Add the 2 for run 2
button2_rows = []   # Add the 2 for run 2

In [53]:
for filepath in glob.glob(f"{unzip_dir}/*.txt"):
    filename = os.path.basename(filepath)

    if "button" in filename.lower():
        # Button file: one column called 'button'
        df = pd.read_csv(filepath, sep='\s+', skiprows=1, names=["button"])
        df["onset"] = df["button"]
        df["duration"] = 0.0
        df["trial_type"] = "button_press"
        button2_rows.append(df[["onset", "duration", "trial_type"]])

    else:
        # Object stimulus files
        df = pd.read_csv(filepath, sep='\s+', skiprows=1, names=["object", "patch", "onset", "prevObject", "dist"])
        df["duration"] = 0.0

        def label_trial(row):
            if pd.isna(row["object"]):
                return "other"
            obj = int(row["object"])
            if obj in [2, 4, 6, 7, 8, 9, 10]:
                return f"object_{obj}"
            elif row["patch"] == 1:
                return "patch_all"
            elif row["patch"] == 0:
                return "no_patch_all"
            else:
                return "other"

        df["trial_type"] = df.apply(label_trial, axis=1)
        stim2_rows.append(df[["onset", "duration", "trial_type"]])

In [54]:
# Merge both types of events
stim2_events = pd.concat(stim2_rows, ignore_index=True)
button2_events = pd.concat(button2_rows, ignore_index=True)
events_2 = pd.concat([stim2_events, button2_events], ignore_index=True).sort_values("onset")

In [55]:
# Final preview
print(events_2["trial_type"].value_counts())

trial_type
object_10       45
object_6        44
object_7        44
object_4        44
object_9        43
object_2        42
object_8        42
button_press    42
Name: count, dtype: int64


In [56]:
events_2

,onset,duration,trial_type
216,3.640,0.0,object_6
261,6.790,0.0,object_4
282,10.850,0.0,object_7
241,12.880,0.0,object_2
127,15.959,0.0,object_8
...,...,...,...
281,1330.369,0.0,object_4
343,1333.976,0.0,button_press
344,1341.965,0.0,button_press
126,1346.679,0.0,object_2


In [57]:
# Save to new _events.tsv
events_2.to_csv("sub04_run2_events.tsv", sep="\t", index=False)

In [58]:
# Downloading them locally
#from google.colab import files

files.download("sub04_run2_events.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Run 3 Subject 04

In [59]:
uploaded = files.upload()

Saving subject4_run3.zip to subject4_run3 (1).zip


In [60]:
# Get the name dynamically
zip_filename = list(uploaded.keys())[0]

In [61]:
# Unzip
unzip_dir = "unzipped_run3"
os.makedirs(unzip_dir, exist_ok=True)
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

print(os.listdir(unzip_dir))  # Sanity check

['Subj_4_run_3_obj_4_patch_0.txt', 'Subj_4_run_3_obj_9_patch_0.txt', 'Subj_4_run_3_obj_8_patch_1.txt', 'Subj_4_run_3_obj_6_patch_0.txt', 'Subj_4_run_3_obj_10_patch_0.txt', 'Subj_4_run_3_button.txt', 'Subj_4_run_3_obj_10_patch_1.txt', 'Subj_4_run_3_obj_6_patch_1.txt', 'Subj_4_run_3_obj_4_patch_1.txt', 'Subj_4_run_3_obj_7_patch_1.txt', 'Subj_4_run_3_obj_7_patch_0.txt', 'Subj_4_run_3_obj_2_patch_1.txt', 'Subj_4_run_3_obj_8_patch_0.txt', 'Subj_4_run_3_obj_2_patch_0.txt', 'Subj_4_run_3_obj_9_patch_1.txt']


In [62]:
stim3_rows = []     # Add the 3 for run 3
button3_rows = []   # Add the 3 for run 3

In [63]:
for filepath in glob.glob(f"{unzip_dir}/*.txt"):
    filename = os.path.basename(filepath)

    if "button" in filename.lower():
        # Button file: one column called 'button'
        df = pd.read_csv(filepath, sep='\s+', skiprows=1, names=["button"])
        df["onset"] = df["button"]
        df["duration"] = 0.0
        df["trial_type"] = "button_press"
        button3_rows.append(df[["onset", "duration", "trial_type"]])

    else:
        # Object stimulus files
        df = pd.read_csv(filepath, sep='\s+', skiprows=1, names=["object", "patch", "onset", "prevObject", "dist"])
        df["duration"] = 0.0

        def label_trial(row):
            if pd.isna(row["object"]):
                return "other"
            obj = int(row["object"])
            if obj in [2, 4, 6, 7, 8, 9, 10]:
                return f"object_{obj}"
            elif row["patch"] == 1:
                return "patch_all"
            elif row["patch"] == 0:
                return "no_patch_all"
            else:
                return "other"

        df["trial_type"] = df.apply(label_trial, axis=1)
        stim3_rows.append(df[["onset", "duration", "trial_type"]])

In [64]:
# Merge both types of events
stim3_events = pd.concat(stim3_rows, ignore_index=True)
button3_events = pd.concat(button3_rows, ignore_index=True)
events_3 = pd.concat([stim3_events, button3_events], ignore_index=True).sort_values("onset")

In [65]:
# Final preview
print(events_3["trial_type"].value_counts())

trial_type
object_6        49
object_9        46
object_10       45
object_4        45
object_8        43
object_7        43
object_2        43
button_press    42
Name: count, dtype: int64


In [66]:
events_3

,onset,duration,trial_type
137,1.260,0.0,object_6
206,4.342,0.0,object_7
23,8.400,0.0,object_9
250,13.440,0.0,object_8
314,17.083,0.0,button_press
...,...,...,...
160,1331.629,0.0,object_6
290,1333.658,0.0,object_2
354,1338.258,0.0,button_press
355,1343.403,0.0,button_press


In [67]:
# Save to new _events.tsv
events_3.to_csv("sub04_run3_events.tsv", sep="\t", index=False)

In [68]:
# Downloading them locally
#from google.colab import files

files.download("sub04_run3_events.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>